### **Advanced AI Coursework**
*B716460*

Aims for this report:
* Results from a range of different parameters
  * Results from a range of varying epochs for its training process.
  * Adjustments to its given optmiser, such as; choice of optmisers and its given adjustments e.g., learning rate.
* Implementation of a Convolutional Neural Network.
* Implementation of a Recurring Neural Network.
* Produce varying comparisons of the two architectures and how they differ.

## Dataset & Image Preprocessing

Get the CIFAR-10 dataset from the Keras API, and convert the downloaded dataset labels to a binary class matrix for faster processing.

Where it states; 
```python
(training_images, training_labels),(testing_images, testing_labels) = cifar10.load_data()
```
All necessary testing and training images, along with their associated labels to represent its particular class (in this case there are 10 classes), 

In [4]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
mnist = tf.keras.datasets.mnist
cifar10 = tf.keras.datasets.cifar10

(training_images, training_labels),(testing_images, testing_labels) = mnist.load_data()

training_images = training_images.reshape(list(training_images.shape) + [1])
testing_images = testing_images.reshape(list(testing_images.shape) + [1])
#Reshape the data to have a 4th dimension for compatibility with tensorflow.

training_labels = tf.keras.utils.to_categorical(training_labels)
testing_labels = tf.keras.utils.to_categorical(testing_labels)

Split the training datase

Create a Keras API sequential neural network model. This sequential model follows the AlexNet structure but has
been tweaked to conform with the CIFAR-10's dataset image size.

[AlexNet Source Code](https://medium.com/analytics-vidhya/alexnet-tensorflow-2-1-0-d398b7c76cf)

Original model was taken from this article and modified to adjust to the shape of the CIFAR-10 dataset images (32x32).

[![Illustrated model](images/alexnet.jpg)](https://www.mdpi.com/2072-4292/9/8/848/htm)

In [6]:

model = tf.keras.models.Sequential([
    # 1st layer
  tf.keras.layers.Conv2D(96, (11,11),strides=(4,4), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(2, strides=(2,2)),
    # 2nd layer
  tf.keras.layers.Conv2D(256, (11,11),strides=(1,1), activation='relu',padding="same"),
  tf.keras.layers.BatchNormalization(),
     # 3rd layer
  tf.keras.layers.Conv2D(384, (3,3),strides=(1,1), activation='relu',padding="same"),
  tf.keras.layers.BatchNormalization(),
    # 4th layer
  tf.keras.layers.Conv2D(384, (3,3),strides=(1,1), activation='relu',padding="same"),
  tf.keras.layers.BatchNormalization(),
    # 5th layer
  tf.keras.layers.Conv2D(256, (3, 3), strides=(1, 1), activation='relu',padding="same"),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(2, strides=(2, 2)),
  # To Flatten layer
  tf.keras.layers.Flatten(),
  # To FC layer 1
  tf.keras.layers.Dense(4096, activation='relu'),
  #To FC layer 2
  tf.keras.layers.Dense(4096, activation='relu'),
  tf.keras.layers.Dense(testing_labels.shape[1], activation='softmax')
])

In [7]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, 
    beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam')
# Source: https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam

loss_function = tf.keras.losses.CategoricalCrossentropy()
# Source: https://www.tensorflow.org/api_docs/python/tf/keras/losses/CategoricalCrossentropy

max_epochs = 10

Compile the model and print a summary results list of the given model.

In [8]:
model.compile(optimizer=optimizer,
              loss=loss_function,
              metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 5, 5, 96)          11712     
_________________________________________________________________
batch_normalization_10 (Batc (None, 5, 5, 96)          384       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 2, 2, 96)          0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 2, 2, 256)         2973952   
_________________________________________________________________
batch_normalization_11 (Batc (None, 2, 2, 256)         1024      
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 2, 2, 384)         885120    
_________________________________________________________________
batch_normalization_12 (Batc (None, 2, 2, 384)        

The Model.fit method adjusts the model parameters to minimize the loss.

In [9]:
history = model.fit(training_images, training_labels, epochs=max_epochs, validation_data=(testing_images, testing_labels))

Epoch 1/10
1875/1875 [==============================] - 56s 26ms/step - loss: 0.5339 - accuracy: 0.8617 - val_loss: 0.1251 - val_accuracy: 0.9693
Epoch 2/10
1875/1875 [==============================] - 45s 24ms/step - loss: 0.1484 - accuracy: 0.9630 - val_loss: 0.1351 - val_accuracy: 0.9687
Epoch 3/10
1875/1875 [==============================] - 50s 27ms/step - loss: 0.1104 - accuracy: 0.9726 - val_loss: 0.0833 - val_accuracy: 0.9791
Epoch 4/10
1875/1875 [==============================] - 49s 26ms/step - loss: 0.0814 - accuracy: 0.9787 - val_loss: 0.0764 - val_accuracy: 0.9786
Epoch 5/10
1875/1875 [==============================] - 50s 27ms/step - loss: 0.0685 - accuracy: 0.9822 - val_loss: 0.0787 - val_accuracy: 0.9822
Epoch 6/10
1875/1875 [==============================] - 50s 27ms/step - loss: 0.0553 - accuracy: 0.9863 - val_loss: 0.0567 - val_accuracy: 0.9844
Epoch 7/10
1875/1875 [==============================] - 46s 25ms/step - loss: 0.0519 - accuracy: 0.9861 - val_loss: 0.0649 -

Plotting the models data to represent the loss value againist its training data and the validation data.

In [9]:
loss_train = history.history['loss']
loss_val = history.history['val_loss']
epochs = range(0, max_epochs)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

ValueError: x and y must have same first dimension, but have shapes (8,) and (10,)

In [ ]:
loss_train = history.history['acc']
loss_val = history.history['val_acc']
epochs = range(0, max_epochs)
plt.plot(epochs, loss_train, 'g', label='Training accuracy')
plt.plot(epochs, loss_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

With the Alexnet implementation complete and results shown, now are implementing a Graph based neural network, and will perform similar adjustments to better understand their differences.